In [38]:
import pickle
import json
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import sent_tokenize

In [3]:
with open('articles.pkl', 'rb') as fp:
    articles = pickle.load(fp)

In [77]:
def filt(x):
    x = x.replace('=', '').replace('\t', ' ').replace('\n', '')
    sents = sent_tokenize(str(x.encode('ascii', 'ignore')))
    return '<p><s>' + "</s><s>".join(sents) + '</s></p>'

def head(s):
    def _(s):
        if type(s) == dict:
            return s.get('print_headline', s.get('main', s.get('content_kicker', '')))

        return s
    return filt(_(s))

bare = [(head(a['headline']), filt(a['snippet'])) for a in articles if a['snippet']]
important = [('abstract=' + head(a['headline']), 'article=' + filt(a['snippet'])) for a in articles if a['snippet']]

In [78]:
len(articles), len(important)

(86411, 83809)

In [79]:
important[0]

("abstract=<p><s>b'With Bradley Out,cubs Bring In Byrd'</s></p>",
 "article=<p><s>b'The Cubs, who had been searching for a center fielder, agreed Thursday to a three-year, $15 million contract with Marlon Byrd....'</s></p>")

In [80]:
train, test = train_test_split(important)

In [81]:
with open('articles_test.txt', 'w') as fp:
    fp.writelines(['\t'.join(pair) + '\n' for pair in test])
    
with open('articles_train.txt', 'w') as fp:
    fp.writelines(['\t'.join(pair) + '\n' for pair in train])

In [82]:
all_text = [' '.join(a) for a in bare]
cv = CountVectorizer(token_pattern='(?:(?u)\\b\\w\\w+\\b)|(?:</?\\w>)')
mat = cv.fit_transform(all_text)

In [83]:
vocab = zip(cv.get_feature_names(), mat.toarray().sum(axis=0))
vocab = sorted(vocab, key=lambda x: x[1], reverse=True)

In [84]:
with open('vocab.txt', 'w') as fp:
    fp.writelines([' '.join(map(str, pair)) + '\n' for pair in vocab])